### Setup

In [17]:
import pandas as pd
import ast
import os

In [18]:
local = pd.read_csv('datasets/territory_codes_local.csv')
foreign = pd.read_csv('datasets/territory_codes_foreign.csv')
codes = (
    pd.concat([local, foreign])
    .rename(columns={'name': 'freguesia'})
    .reset_index(drop=True)
)

results = pd.DataFrame()
metadata = pd.DataFrame()

### Methods

In [19]:
# Returns resuts table df
def getResultsTable(file):
    resultsParty = file.loc['resultsParty', 'currentResults']

    results = (
        pd.DataFrame(ast.literal_eval(resultsParty))
        .drop(['absoluteMajority', 'constituenctyCounter', 'imageKey', 'mandates', 'presidents'], axis=1)
        .set_index('acronym')
    )

    return results

# Returns metadata df
def getMetadata(file):
    cr = pd.Series(
        file['currentResults']
        .drop(['availableMandates', 'compensation', 'displayMessage', 'hasNoVoting', 'resultsParty'
            , 'tie', 'tieMessage', 'totalBoycotts', 'totalForeignBoycotts', 'totalLocalBoycotts'
            , 'totalMandates', 'totalParishesApproved'])
    )

    metadata = (
        file.drop(['currentResults', 'displayMessage', 'foreignCounterMessage', 'nationalCounterMessage', 'previousResults'
                    , 'refreshTimeout', 'territoryMessage', 'time', 'timeOnServer', 'warningMessage'], axis=1)
        .loc['availableMandates', :]
    )

    metadata = (
        pd.concat([metadata, cr])
        .to_frame(name='metadata')
        .T
        .set_index('territoryKey')
    )

    return metadata

# Returns results df
def getResults(file):
    r = getResultsTable(file)
    metadata = getMetadata(file)
    r['tkey'] = metadata.index[0]
    r['parish'] = metadata['territoryFullName'].iloc[0] #! fix To access a value by position, use ser.iloc[pos]

    r = r.reset_index().set_index(['parish', 'acronym'])

    return r

# Creates directories
def mkdir():
        path = '/datasets/Results/' 

        working_dir = os.getcwd()
        newpath = working_dir + path

        if not os.path.exists(newpath):
            os.makedirs(newpath)

In [20]:
for i, row in codes.iterrows():
    loc = row['territoryKey'].split('-')[0].capitalize()
    file = (
        pd.read_csv('datasets/' + loc + '/' + row['distrito'] + '/' + row['concelho'] + '/' + row['freguesia'] + '.csv')
        .set_index('index')
    )

    results = pd.concat([results, getResults(file)])
    metadata = pd.concat([metadata, getMetadata(file)])

In [21]:
mkdir()

results.to_csv('datasets/Results/results.csv')
metadata.to_csv('datasets/Results/metadata.csv')